In [ ]:
import kagglehub

path = kagglehub.dataset_download("barelydedicated/bank-customer-churn-modeling")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/bank-customer-churn-modeling


In [ ]:
import pandas as pd

dataset_path = "/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv"
df = pd.read_csv(dataset_path)

print(df.head())

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

In [ ]:
df.shape

(10000, 14)

In [ ]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [ ]:
x = df[['CreditScore','Age','Tenure','Balance','NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
y = df['Exited']

In [ ]:
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,42,2,0.00,1,1,1,101348.88
1,608,41,1,83807.86,1,0,1,112542.58
2,502,42,8,159660.80,3,1,0,113931.57
3,699,39,1,0.00,2,0,0,93826.63
4,850,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64
9996,516,35,10,57369.61,1,1,1,101699.77
9997,709,36,7,0.00,1,0,1,42085.58
9998,772,42,3,75075.31,2,1,0,92888.52


In [ ]:
y.value_counts()

,count
Exited,
0,7963
1,2037


Normalization using standard derivation

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [ ]:
x_scaled

array([[-0.32622142,  0.29351742, -1.04175968, ...,  0.64609167,
         0.97024255,  0.02188649],
       [-0.44003595,  0.19816383, -1.38753759, ..., -1.54776799,
         0.97024255,  0.21653375],
       [-1.53679418,  0.29351742,  1.03290776, ...,  0.64609167,
        -1.03067011,  0.2406869 ],
       ...,
       [ 0.60498839, -0.27860412,  0.68712986, ..., -1.54776799,
         0.97024255, -1.00864308],
       [ 1.25683526,  0.29351742, -0.69598177, ...,  0.64609167,
        -1.03067011, -0.12523071],
       [ 1.46377078, -1.04143285, -0.35020386, ...,  0.64609167,
        -1.03067011, -1.07636976]])

Cross_Validation

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y,random_state=0, test_size=0.25)

In [ ]:
x.shape

(10000, 8)

In [ ]:
x_test.shape

(2500, 8)

In [ ]:
x_train.shape

(7500, 8)

Create Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(hidden_layer_sizes=(100,100,100), random_state=0, max_iter=100,activation='relu')

In [ ]:
ann.fit(x_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=100, random_state=0)

In [ ]:
y_pred = ann.predict(x_test)

In [ ]:
from sklearn import metrics
y_test.value_counts()

,count
Exited,
0,1991
1,509


In [ ]:
print("Confusion Matrix : ")
cs = metrics.confusion_matrix(y_test, y_pred)
print(cs)

Confusion Matrix : 
[[1830  161]
 [ 282  227]]


In [ ]:
print(metrics.accuracy_score(y_test, y_pred))

0.8228


In [ ]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89      1991
           1       0.59      0.45      0.51       509

    accuracy                           0.82      2500
   macro avg       0.73      0.68      0.70      2500
weighted avg       0.81      0.82      0.81      2500



In [ ]:
### Balancing the data by over_sampling (we can use under samling also but data get reduced so used over_sampling)
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state = 0)

In [ ]:
x_res, y_res = ros.fit_resample(x,y)
y_res.value_counts()

,count
Exited,
1,7963
0,7963


Normalization using standard derivation

In [ ]:
balanced_x_scaled = scaler.fit_transform(x_res)
balanced_x_scaled

array([[-0.29877723,  0.08418894, -1.01840607, ...,  0.6512495 ,
         1.08223556,  0.00817382],
       [-0.4103938 , -0.01032629, -1.36135608, ..., -1.53550983,
         1.08223556,  0.20261687],
       [-1.48597169,  0.08418894,  1.03929402, ...,  0.6512495 ,
        -0.92401325,  0.22674468],
       ...,
       [-0.84671313,  1.02934128,  0.01044398, ...,  0.6512495 ,
        -0.92401325,  1.28878188],
       [-0.96847667,  0.65128034, -0.67545605, ..., -1.53550983,
         1.08223556, -1.21851316],
       [-1.5874413 ,  0.74579558,  1.03929402, ...,  0.6512495 ,
         1.08223556,  1.42417217]])

Cross Validation

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(balanced_x_scaled, y_res, random_state=0, test_size=0.25)

In [ ]:
x_res.shape

(15926, 8)

In [ ]:
x_test.shape

(3982, 8)

In [ ]:
x_train.shape

(11944, 8)

Creating Neural Networks

In [ ]:
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(hidden_layer_sizes=(100,100,100), random_state=0, max_iter=100,activation='relu')

In [ ]:
ann.fit(x_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=100, random_state=0)

In [ ]:
y_pred = ann.predict(x_test)

In [ ]:
y_test.value_counts()

,count
Exited,
1,2001
0,1981


In [ ]:
print("Confusion Matrix : ")
cs = metrics.confusion_matrix(y_test, y_pred)
print(cs)

Confusion Matrix : 
[[1630  351]
 [ 186 1815]]


In [ ]:
print(metrics.accuracy_score(y_test, y_pred))

0.865143144148669


In [ ]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86      1981
           1       0.84      0.91      0.87      2001

    accuracy                           0.87      3982
   macro avg       0.87      0.86      0.86      3982
weighted avg       0.87      0.87      0.86      3982

